## Mount Google drive to access images and masks

In [2]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 124kB/s 


In [4]:
#Set the project path 
project_path =  '/content/drive/My Drive/AIML_Delete/Capstone/'

In [5]:
import os
os.chdir(project_path)
path = '.'

In [ ]:
ls

 dicom_image_description.csv              stage_2_detailed_class_info.gsheet
'GCP Credits Request Link - RSNA.txt'     stage_2_sample_submission.csv
 model_weights/                           stage_2_test_images/
 Mridula_Notes.gdoc                       stage_2_train_images/
 output/                                  stage_2_train_images_png/
 Patient_Detail.csv                       stage_2_train_labels.csv
 rsna-pneumonia-detection-challenge.zip   Training_labels_merged.csv
 stage_2_detailed_class_info.csv


In [6]:
TRAIN_IMAGES = os.path.join(project_path + 'stage_2_train_images/')
TEST_IMAGES = os.path.join(project_path + 'stage_2_test_images/')
OUTPUT_DIR = os.path.join(project_path + 'output/')
if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

In [7]:
import pydicom 
from pydicom.data import get_testdata_files
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [8]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import data_utils
import numpy as np
from PIL import Image
from keras.utils.layer_utils import count_params
from tabulate import tabulate

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import ipywidgets as widgets
import pandas as pd
from IPython.display import clear_output
import calendar
import time
import datetime
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model
from keras.layers import Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import random
from tqdm import tqdm

#from generator import DataGenerator


Using TensorFlow backend.


In [9]:
#Read pkl file with the feature engineered data
train_class = pd.read_pickle(project_path+'output/training_df.pkl')
display(train_class.shape, train_class.head())

(30227, 9)

,PatientID,x,y,width,height,Target,number_of_boxes,class,path
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,1,No Lung Opacity / Not Normal,/content/drive/My Drive/AIML_Delete/Capstone/s...
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,1,No Lung Opacity / Not Normal,/content/drive/My Drive/AIML_Delete/Capstone/s...
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,1,No Lung Opacity / Not Normal,/content/drive/My Drive/AIML_Delete/Capstone/s...
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,1,Normal,/content/drive/My Drive/AIML_Delete/Capstone/s...
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,2,Lung Opacity,/content/drive/My Drive/AIML_Delete/Capstone/s...


In [10]:

TRAIN_IMAGES_DIR = os.path.join(project_path + 'stage_2_train_images_png/')
if not os.path.exists(TRAIN_IMAGES_DIR): os.makedirs(TRAIN_IMAGES_DIR)

In [11]:
# Store only paths , classes and targets to another dataframe
print('Prepare a dataframe with paths, classes and targets'); print('--'*40)
path_class_target = train_class[['PatientID', 'path', 'class', 'Target']].copy(deep = True)
path_class_target['path'] = (path_class_target['path']
                             .str.replace('stage_2_train_images', 'stage_2_train_images_png')
                             .str.replace('.dcm', '.png'))
path_class_target.drop_duplicates(inplace = True)
display(path_class_target.shape, path_class_target.nunique())
print('\nDistribution of target and classes')
display(path_class_target['Target'].value_counts())
print()
display(path_class_target['class'].value_counts())

Prepare a dataframe with paths, classes and targets
--------------------------------------------------------------------------------


(26684, 4)

PatientID    26684
path         26684
class            3
Target           2
dtype: int64


Distribution of target and classes


0    20672
1     6012
Name: Target, dtype: int64

No Lung Opacity / Not Normal    11821
Normal                           8851
Lung Opacity                     6012
Name: class, dtype: int64

In [12]:
# Split the data in train, valid and test sets

image_list = list(path_class_target['path'])
random.shuffle(image_list)
val_size = round(len(image_list)/10)

train_size = len(image_list)-val_size

X_train = image_list[:train_size]
X_valid = image_list[train_size:(train_size + val_size)]


In [13]:
# Create training and validation dataframes concatenated woth path and target class 

df_train = (path_class_target.merge(pd.Series(X_train, name = 'path'), 
                                    on = 'path', 
                                    how = 'right')
          .drop(['class', 'PatientID'], axis = 1))

df_valid = (path_class_target.merge(pd.Series(X_valid, name = 'path'), 
                                    on = 'path', 
                                    how = 'right')
          .drop(['class', 'PatientID'], axis = 1))


print('Shape of the dataframes:\nTRAIN:{}\nVALID:{}'.format(df_train.shape, df_valid.shape))

Shape of the dataframes:
TRAIN:(24016, 2)
VALID:(2668, 2)


In [ ]:
#Store train and test dataframes in pkl. Only to be executed ONCE
#df_train.to_pickle(OUTPUT_DIR + 'train_data.pkl')
#df_valid.to_pickle(OUTPUT_DIR + 'valid_data.pkl')


In [14]:
df_train = pd.read_pickle(OUTPUT_DIR + 'train_data.pkl')
df_valid = pd.read_pickle(OUTPUT_DIR + 'valid_data.pkl')
display(df_train.shape, df_valid.shape)

(24016, 2)

(2668, 2)

In [17]:
# Check if there is a class imbalance
print("Count of patients in each class in Training Dataset \n",df_train['Target'].value_counts())
print("Count of patients in each class in Validation Dataset \n",df_valid['Target'].value_counts())

Count of patients in each class in Training Dataset 
 0    18599
1     5417
Name: Target, dtype: int64
Count of patients in each class in Validation Dataset 
 0    2073
1     595
Name: Target, dtype: int64


There is a clear imbalance seen with almost 3 times for patients belonging to the Non-Pneumonia target group

In [26]:
# Correct the imbalance
class_count = df_train['Target'].value_counts()
COUNT_PNEUMONIA=class_count[1]
display(COUNT_PNEUMONIA)
COUNT_NORMAL=class_count[0]
display(COUNT_NORMAL)

5417

18599

In [30]:
initial_bias = np.log([COUNT_PNEUMONIA/COUNT_NORMAL])
initial_bias


24016

In [31]:
weight_for_0 = (1 / COUNT_NORMAL)*(df_train.shape[0])/2.0 
weight_for_1 = (1 / COUNT_PNEUMONIA)*(df_train.shape[0])/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.65
Weight for class 1: 2.22


# Generator for Images

In [32]:
# MODEL IMPORTS
import tensorflow as tf
assert tf.__version__ >= '2.0'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from skimage.transform import resize
from imgaug import augmenters as iaa

import pydicom as dcm, cv2
from PIL import Image
import pandas as pd
import numpy as np
import keras
random_state = 2020


def roc_auc(y_true, y_pred):
    return tf.compat.v1.py_function(roc_auc_score, (y_true, y_pred), tf.double)


import tensorflow.keras.backend as K
def average_precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# Augmenter
augmenter = ImageDataGenerator(preprocessing_function = preprocess_input, 
                               rotation_range = 20, width_shift_range = 0.2,
                               height_shift_range = 0.2, zoom_range = 0.2,
                               horizontal_flip = True)

# Data Generator
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataset_df, img_dir, batch_size = 32, dim = (224, 224), 
                 transform = None, n_channels = 1, shuffle = True, debug = False):
        
        self.dataset_df = dataset_df
        self.paths = self.dataset_df['path']
        self.img_dir = img_dir 
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.debug = debug
        self.transform = transform
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataset_df) / self.batch_size))

    def __getitem__(self,index):
        indexes = self.indexes[index*self.batch_size:(index + 1)*self.batch_size]    

        list_IDs_batch = [self.paths[k] for k in indexes]

        imgs, labels = self.__data_generation(list_IDs_batch)

        if self.debug: return list_IDs_batch, imgs, labels 
        else: return imgs, labels
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.dataset_df))
        if self.shuffle == True: np.random.shuffle(self.indexes)
    
    def __data_generation(self, list_IDs_batch):
        imgs = np.empty((self.batch_size, *self.dim, self.n_channels))
        labels = np.empty((self.batch_size), dtype = int)
        for i, ID in enumerate(list_IDs_batch): 
            imgs[i,] = self.load_img(self.img_dir, ID, self.dim)
            labels[i] = self.get_label(ID)
        return imgs, labels
    
    def load_img(self, img_dir, path, dim):
        img = Image.open(path)
        img = np.asarray(img.convert('RGB'))
        img = img / 255.
        img = cv2.resize(img, dim)
        if not self.transform == None:
            params = self.transform.get_random_transform(img.shape)
            img = self.transform.apply_transform(img, params)
        return img
    
    def get_label(self, path):
        data_df = self.dataset_df.loc[self.dataset_df['path'] == path].values
        return int(data_df[0][1])

# Predictor
def predictor(model, validation_generator):
    y_pred = []
    y_roc = []
    y_true = []
    for i, batch in enumerate(validation_generator):
        imgs = batch[0]
        labels = batch[1]
        for img, label in zip(imgs, labels):
            img = np.expand_dims(img, axis = 0)
            pred = model.predict(img)
            y_roc.append(pred)
            if pred > 0.5: pred = 1
            else: pred = 0
            y_pred.append(pred)
            y_true.append(label)
    return y_true, y_pred, y_roc

# CheXNet Model

In [49]:
#hyper parameter
input_shape = (224, 224, 3)
num_of_class = 1
initializer = tf.keras.initializers.GlorotNormal()


In [34]:
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

In [35]:
# Path to save the model and weights
MODEL_WEIGHTS = os.path.join(project_path + 'model_weights/')
if not os.path.exists(MODEL_WEIGHTS): os.makedirs(MODEL_WEIGHTS)
FINAL_MODEL = 'chexnet_final.h5'


In [41]:
# Using pretrained weights ( Actual CheXNet used weights from imagenet)
# Ref : https://github.com/brucechou1983/CheXNet-Keras
PRETRAINED_WEIGHTS = 'CheXNet_weights241.h5'

In [53]:
img_in = Input(input_shape)              #input of model 
model = DenseNet121(include_top= True , # remove  the 3 fully-connected layers at the top of the network
                weights=MODEL_WEIGHTS + PRETRAINED_WEIGHTS,      # pre train weight         
                input_tensor= img_in,      
                input_shape= input_shape,
                pooling ='avg',
                classes=14) 
sample = int(len(model.layers)/3)
for layer in model.layers[:sample]:
  layer.trainable=True
for layer in model.layers[sample:]:
  layer.trainable=False

x = model.output  

#x= GlobalAveragePooling2D()(x)
#x= BatchNormalization()(x)
#x= Dropout(0.5)(x)
#x= Dense(1024,activation='relu')(x) 
#x= Dense(224,activation='relu')(x) 
#x= BatchNormalization()(x)
#x= Dropout(0.5)(x)
#predictions = Dense(num_of_class,activation="sigmoid", name="predictions")(x)    # fully connected layer for predict class 
output_layer = Dense(1, activation = 'sigmoid', kernel_initializer = initializer)(x)
#x1=tf.keras.layers.Reshape(((224, 224)))
#model = Model(inputs=img_in, outputs=predictions,name = "CheXNet")
model = Model(inputs=img_in, outputs=output_layer,name = "CheXNet")

In [54]:
model.summary()

Model: "CheXNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_22 (ZeroPadding2 (None, 230, 230, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_22[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

## Model Execution


In [ ]:
batch_size = 32
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[keras.metrics.binary_accuracy,'accuracy', average_precision])
train_generator = DataGenerator(dataset_df = df_train, img_dir = TRAIN_IMAGES_DIR,
                                batch_size = batch_size, dim = (224, 224),
                                transform = augmenter, n_channels = 3, shuffle = True)

validation_generator = DataGenerator(dataset_df = df_valid, img_dir = TRAIN_IMAGES_DIR,
                                     batch_size = batch_size, dim = (224, 224),
                                     transform = ImageDataGenerator(preprocessing_function = preprocess_input), 
                                     n_channels = 3, shuffle = True)
history =model.fit_generator(generator=train_generator,
                    epochs = 3,
                    validation_data=validation_generator,
                    steps_per_epoch=(24016//(batch_size**2))*2,
                    validation_steps = (2668//(batch_size**2))*2,
                    class_weight=class_weight,
                    use_multiprocessing=True,
                    workers = 8)

Epoch 1/3
46/46 [==============================] - 826s 18s/step - loss: 0.6813 - binary_accuracy: 0.6732 - accuracy: 0.6732 - average_precision: 0.2224 - val_loss: 0.7000 - val_binary_accuracy: 0.5938 - val_accuracy: 0.5938 - val_average_precision: 0.3033
Epoch 2/3


Process Keras_worker_ForkPoolWorker-13:
Process Keras_worker_ForkPoolWorker-16:
Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-9:
Process Keras_worker_ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-14:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-22:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in

In [ ]:
# Saving the weights


model.save(MODEL_WEIGHTS + FINAL_MODEL)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [ ]:
import math
validation_generator = DataGenerator(dataset_df = df_valid, img_dir = TRAIN_IMAGES_DIR,
                                     batch_size = batch_size * 2, dim = (224, 224),
                                     transform = ImageDataGenerator(preprocessing_function = preprocess_input), 
                                     n_channels = 3, shuffle = False)

loss, accuracy, ap = model.evaluate_generator(generator = validation_generator, 
                                              steps = 2668//(batch_size**2))
print(f'Loss: {round(loss, 3)}, Accuracy: {round(float(accuracy), 3)}, AP: {round(float(ap), 3)}')